In [108]:
from util.api_and_repo_link import link_me_up
link_me_up()
from config.environment import ENVIRONMENT
ENVIRONMENT.set_development()from util.api_and_repo_link import link_me_up
link_me_up()
from config.environment import ENVIRONMENT
ENVIRONMENT.set_development()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
C:\Users\Thomas.fuller\NotDrive\Repos\airflow_pipelines\dags
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
now you just need to set_development()


In [109]:

from multithread_uploader import upload_all_these_records
from oliverbonas_source_dags.apis.api_netsuite import APIConfig_netsuite
api = APIConfig_netsuite()
from util.get_ns_number import get_ns_number
from util.zeep_to_df import complete_zeep_df
import pandas
import random
from datetime import datetime, timedelta

In [73]:
StringCustomFieldRef = api.client.get_type(f"ns0:StringCustomFieldRef")
SalesOrder = api.client.get_type("ns19:SalesOrder")
RecordRef = api.client.get_type("ns0:RecordRef")
SalesOrderItemList = api.client.get_type("ns19:SalesOrderItemList")
SalesOrderItem = api.client.get_type("ns19:SalesOrderItem")
LocationSearchBasic = api.client.get_type("ns5:LocationSearchBasic")
DepartmentSearchBasic = api.client.get_type("ns5:DepartmentSearchBasic")
Address = api.client.get_type("ns5:Address")
GetAllRecord = api.client.get_type("ns0:GetAllRecord")
CustomerSearchBasic = api.client.get_type("ns5:CustomerSearchBasic")
CustomFieldList = api.client.get_type("ns0:CustomFieldList")
CustomRecordRef = api.client.get_type("ns0:CustomRecordRef")
CustomFieldRef = api.client.get_type("ns0:CustomFieldRef")
ListOrRecordRef = api.client.get_type("ns0:ListOrRecordRef")
BooleanCustomFieldRef = api.client.get_type("ns0:BooleanCustomFieldRef")
TransactionSearchBasic = api.client.get_type("ns5:TransactionSearchBasic")
SelectCustomFieldRef = api.client.get_type("ns0:SelectCustomFieldRef")
IntercoStatus = api.client.get_type("ns6:IntercoStatus")
get_ns_number(api, "IntercoStatus")

failed for ns 0
failed for ns 1
failed for ns 2
failed for ns 3
failed for ns 4
failed for ns 5
IntercoStatus = api.client.get_type("ns6:IntercoStatus")


# downloads before uploads (up and down)

In [ ]:
response = api.service_proxy.search(
    searchRecord = LocationSearchBasic(),
    _soapheaders = api.build_soap_headers(),
)

locations = response.body.searchResult.recordList.record

In [ ]:
def location_fixup(location_str):
    print(location_str.lower())
    if location_str.lower() in ("so warehouse", "2-warehouse"):
        return "warehouse"
    elif location_str.lower().endswith("_dnu"):
        # dnu = do not use
        return None
    else:
        return location_str.replace(" -","-").replace("- ", "-").strip().lower()


location_map = {
    location_fixup(location.name): location.internalId
    for location in locations
}
pandas.DataFrame(location_map.items(), columns=["name", "internalId"])

In [ ]:
response = api.service_proxy.search(
    searchRecord = DepartmentSearchBasic(),
    _soapheaders = api.build_soap_headers(),
)

departments = response.body.searchResult.recordList.record

In [ ]:
department_map = {
    department.name.lower()[:3]: department.internalId
    for department in departments
}
pandas.DataFrame(department_map.items(), columns=["name", "internalId"])

In [ ]:
response = api.service_proxy.getAll(
    record=GetAllRecord(recordType="currency"),
    _soapheaders = api.build_soap_headers(),
)
currencies = response.body.getAllResult.recordList.record

In [ ]:
currency_map = {
    currency.name: currency.internalId
    for currency in currencies
}
pandas.DataFrame(currency_map.items(), columns=["name", "internalId"])

In [82]:
IntercoStatus(
    1
)

'1'

### remember coding can be hard sometimes

In [ ]:
STATUSMAP = {
    "Fulfillmement": "_pendingFulfillment",
}

In [ ]:
class_map = {
 "Marketplace": 2,
    "eBay": 5,
    "Next": 7,
    "Platform +": 6,
    "SO": 4,
    "Stores": 3,
    "Test Class 1234": 11,
    "Web": 1,
    "EUR Web Store": 10,
    "GBP Web Store": 9,
    "USD Web Store": 8,
}

# def def def

In [ ]:
def trandate_fixup(trandate_str):
    if type(trandate_str) == str:
        if "/" in trandate_str:
            trandate = datetime.strptime(trandate_str, "%d/%m/%Y")
        else:
            trandate = datetime.strptime(trandate_str, "%Y-%m-%d")
    else:
        trandate = trandate_str

    return trandate.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

# test comes first, that's how it goes in life

In [ ]:
df_original = pandas.concat(
    [pandas.read_csv("test_upload/sale_test.csv"),
     #pandas.read_csv("SLS_NS_20220101_20230430_2.csv"),
     ]
)

In [ ]:
df_original.Location

In [ ]:
df = df_original.loc[df_original.ExternalID.notna()]
df.Location = df.Location.apply(location_fixup)
df.Department = df.Department.astype(int).astype(str).str[:3].str.zfill(3)
df.Department.value_counts()

In [ ]:


api.service_proxy.search(
     searchRecord=TransactionSearchBasic(
            lastModifiedDate={
                "operator": "within",
                "searchValue": datetime.now() - timedelta(days = 1),
                "searchValue2": datetime.now(),
            },
            recordType={
                "operator": "contains",
                # why in the world is it salesOrder and not SalesOrder
                "searchValue": "salesOrder",
            },
    ),
    _soapheaders = api.build_soap_headers(),
)

In [107]:
PREFIX = "tom_f__DATA_TEAM__customer_for_transaction__"
ok = ["003", "004", "005", "006", "007", "008", "009", "010", "011", "012",
      "013", "014", "015", "016", "017", "018", "019", "020",
      "021", "022", "023", "024", "025", "026", "027", "028", "029", "030",
      "031", "032", "033", "034", "035", "036", "037", "038", "039", "040",
      "041", "042", "043", "044", "045", "046", "047", "048", "049", "050",
      "051", "052", "053", "054", "055", "056", "057", "058", "059", "060",]

r = df.iloc[0]
items = []
for yeah in ok:
    item = SalesOrder(
    subsidiary=RecordRef(internalId=2), # only 4 works
    externalId = "testsub6",
    #department = RecordRef(name=department_map[r.Department.lower()]),
    #location = RecordRef(internalId=location_map[r.Location.lower()]),
    entity = RecordRef(
        #internalId=578154,customer_human_881003451
        externalId="customer_human_881003451",
        #externalId="thirteen alex made this",#PREFIX + yeah,
        type="customer",
        #externalId=str(r.Customer).replace(".0", "")
    ),
    #location=RecordRef(internalId=location_map[r.Location]),
    #department=RecordRef(internalId=department_map[r.Department]),
    #opportunity=RecordRef(internalId=1),
    #discountItem
    #discountRate="-0.1",
    #intercoStatus=IntercoStatus("_rejected"),
    #currency = RecordRef(internalId=1),
    #trandate="2021-10-01T00:00:00.000Z",
    memo="meommeomeomeomeomeomeomeome",
    otherRefNum="this should link to the purchase order",
    billingAddress = Address(
        city = r.billCity,
        state = r.billState,
        zip = r.billZip,
        addr1 = r.billAddr1,
        addr2 = r.billAddr2,
        addressee = r.billAddressee,
    ),
    #orderStatus = "_closed",#STATUSMAP[r.orderstatus],
    email = r.email,
    shippingAddress = Address(
        city = r.shipCity,
        state = r.shipState,
        zip = r.shipZip,
        addr1 = r.shipAddr1,
        addr2 = r.shipAddr2,
        addressee = r.shipaddressee,
    ),
    exchangeRate = r.exchangerate,
    itemList = SalesOrderItemList(
        item = [
            SalesOrderItem(
                item = RecordRef(
                    externalId = str(int(r.itemLine_item))
                ),
                description = r.itemLine_description,
                isTaxable = r.itemLine_isTaxable,
                quantity = r.itemLine_quantity,
                amount = 77,
                taxRate1 = r.taxrate,
                isClosed=True
                #amount = r.itemLine_amount,
            )
        ]
    ),

    tranDate = trandate_fixup(datetime(2027, 9, 19)),
    #datetime.strptime(r.trandate, "%d/%m/%Y").strftime("%Y-%m-%dT%H:%M:%S.%fZ"),
    customFieldList =CustomFieldList(
        customField=[
            BooleanCustomFieldRef(
                scriptId="custbody_15699_exclude_from_ep_process",
                value=True
            ),
            SelectCustomFieldRef(
                scriptId="cseg_atlas_cost_ctr",
                value=ListOrRecordRef(
                    internalId=1,
                    typeId=456,
                )
            ),
            StringCustomFieldRef(
                scriptId="custbody_ob_pw_discount_code",
                value="free form text - that means you can type whatever you like here"
            )
        ]
    )
    )
    if pandas.notna(r.taxrate):
        item.taxRate = r.taxrate
    
    # class is a python keyword
    item["class"] = RecordRef(internalId=class_map["Stores"])
    items.append(item)
    break
upload_all_these_records(items, api, "addList")

> [1] records in this chunk

THREADS NEEDED: 1 

thread (0) spawned
+++++++++ new thread (0) +++++++++
# thread (0) total [1] records [1]  success [1] failure [0]
0:00:07.945724
+++++++++ thread completed (0) +++++++++


# below is where the magic happens, the upload

In [ ]:

orders = []

for pos, r in df_original.iterrows():
    if pandas.isna(r.Customer):
        continue
    #print(str(r.Customer).replace(".0", ""))
    print(pos, r.itemLine_quantity)
    
    item = SalesOrder(
        subsidiary=RecordRef(internalId=[random.choice()]), # only 4 works
        externalId = "saying_hi_to_patchworks" + str(pos),#r.ExternalID + "",
        #department = RecordRef(name=department_map[r.Department.lower()]),
        #location = RecordRef(internalId=location_map[r.Location.lower()]),
        entity = RecordRef(
            internalId=578154,
            #externalId=str(r.Customer).replace(".0", "")
        ),
        currency = RecordRef(internalId=1),
        billingAddress = Address(
            city = r.billCity,
            state = r.billState,
            zip = r.billZip,
            addr1 = r.billAddr1,
            addr2 = r.billAddr2,
            addressee = r.billAddressee,
        ),
        orderStatus = STATUSMAP[r.orderstatus],
        email = r.email,
        shippingAddress = Address(
            city = r.shipCity,
            state = r.shipState,
            zip = r.shipZip,
            addr1 = r.shipAddr1,
            addr2 = r.shipAddr2,
            addressee = r.shipaddressee,
        ),
        exchangeRate = r.exchangerate,
        itemList = SalesOrderItemList(
            item = [
                SalesOrderItem(
                    item = RecordRef(
                        externalId = str(int(r.itemLine_item))
                    ),
                    description = r.itemLine_description,
                    isTaxable = r.itemLine_isTaxable,
                    quantity = r.itemLine_quantity,
                    amount = 77,
                    taxRate1 = r.taxrate,
                    #amount = r.itemLine_amount,
                )
            ]
        ),
        
        tranDate = trandate_fixup(r.trandate),
        #datetime.strptime(r.trandate, "%d/%m/%Y").strftime("%Y-%m-%dT%H:%M:%S.%fZ"),
        
    )
    if pandas.notna(r.taxrate):
        item.taxRate = r.taxrate
    
    # class is a python keyword
    item["class"] = RecordRef(name=r.Class)
    orders.append(item)
    break

In [ ]:
#TypeError: IntercoStatus() takes exactly 1 argument (2 given). Simple types expect only a single value argument